In [22]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering, DBSCAN

import scipy
import seaborn as sns


from sklearn.decomposition import PCA
from sklearn import datasets, metrics
from sqlalchemy import create_engine

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "heartdisease"

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

df = pd.read_sql_query("select * from heartdisease", con=engine)
engine.dispose()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
X = df.iloc[:, :13]
y = df.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace="?", value=0)

# Binarize `y` so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 0, 1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
scaler = StandardScaler()
scaled = scaler.fit_transform(X)
scaled_df = pd.DataFrame(scaled, columns=X.columns)
scaled_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.948726,0.686202,-2.251775,0.757525,-0.264900,2.394438,1.016684,0.017197,-0.696631,1.087338,2.274579,-0.711131,0.659133
1,1.392002,0.686202,0.877985,1.611220,0.760415,-0.417635,1.016684,-1.821905,1.435481,0.397182,0.649113,2.504881,-0.865426
2,1.392002,0.686202,0.877985,-0.665300,-0.342283,-0.417635,1.016684,-0.902354,1.435481,1.346147,0.649113,1.432877,1.167319
3,-1.932564,0.686202,-0.165268,-0.096170,0.063974,-0.417635,-0.996749,1.637359,-0.696631,2.122573,2.274579,-0.711131,-0.865426
4,-1.489288,-1.457296,-1.208521,-0.096170,-0.825922,-0.417635,1.016684,0.980537,-0.696631,0.310912,-0.976352,-0.711131,-0.865426


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 1. Apply GMM to the heart disease dataset by setting n_components=2. Get ARI and silhouette scores for your solution and compare it with those of the k-means and hierarchical clustering solutions that you implemented in the previous checkpoint assignments. Which algorithm performs best?

In [28]:
gauss = GaussianMixture(n_components=2)
gauss.fit(scaled_df)

print(f"Gauss scaled centroid points with 2 clusters:\n {gauss.means_}")

centroids = scaler.inverse_transform(gauss.means_)
centroids_df = pd.DataFrame(centroids, columns=scaled_df.columns)
print("\n Gauss unscaled centroid points with 2 clusters")
display(centroids_df.style.background_gradient())

clusters = gauss.fit_predict(scaled_df)


print(
    "Adjusted Rand Index of the GMM solution: {}".format(
        metrics.adjusted_rand_score(y, clusters)
    )
)
print(
    "The silhoutte score of the GMM solution: {}".format(
        metrics.silhouette_score(X, clusters, metric="euclidean")
    )
)

Gauss scaled centroid points with 2 clusters:
 [[ 0.32278577  0.34253754  0.70353557  0.07234044  0.07213556  0.10642273
   0.16477666 -0.59284025  0.66027532  0.55358459  0.36283028  0.65659998
   0.75796796]
 [-0.21979453 -0.2332441  -0.47905851 -0.04925878 -0.04911927 -0.07246644
  -0.11220138  0.40368274 -0.44960131 -0.37695238 -0.24706204 -0.44709866
  -0.51612316]]

 Gauss unscaled centroid points with 2 clusters


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,57.351677,0.839671,3.832783,132.960840,250.421858,0.186360,1.153776,136.068435,0.636414,1.681296,1.823876,1.275864,6.194487
1,52.455576,0.571053,2.699219,130.824259,244.154025,0.122745,0.878646,158.826254,0.115861,0.602657,1.448665,0.246298,3.687352


Adjusted Rand Index of the GMM solution: 0.18389186035089963
The silhoutte score of the GMM solution: 0.03924007666635825


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Gauss mixture doesnt have great scores.

In [29]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(scaled_df)

print(f"kmeans scaled centroid points with 2 clusters:\n {gauss.means_}")

centroids = scaler.inverse_transform(kmeans.cluster_centers_)
centroids_df = pd.DataFrame(centroids, columns=scaled_df.columns)
print("kmeans unscaled centroid points with 2 clusters")
display(centroids_df.style.background_gradient())

clusters = kmeans.fit_predict(scaled_df)


print(
    "Adjusted Rand Index of the kmeans solution: {}".format(
        metrics.adjusted_rand_score(y, clusters)
    )
)
print(
    "The silhoutte score of the kmeans solution: {}".format(
        metrics.silhouette_score(X, clusters, metric="euclidean")
    )
)

kmeans scaled centroid points with 2 clusters:
 [[-0.06385369 -0.10184842 -0.2675476  -0.0451155  -0.04271068 -0.01787913
  -0.05912091  0.2633984  -0.69663055 -0.20078481 -0.17955539 -0.10156047
  -0.2202288 ]
 [ 0.13157731  0.20986947  0.5513102   0.09296528  0.08800989  0.03684183
   0.12182491 -0.54276033  1.43548113  0.4137384   0.36999292  0.20927611
   0.4538048 ]]
kmeans unscaled centroid points with 2 clusters


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,58.228070,0.815789,3.771930,134.964912,254.666667,0.192982,1.219298,132.947368,0.675439,1.874561,1.964912,1.254386,6.166667
1,52.153439,0.597884,2.788360,129.714286,241.883598,0.121693,0.851852,159.656085,0.116402,0.535979,1.380952,0.306878,3.820106


Adjusted Rand Index of the kmeans solution: 0.4380857727169879
The silhoutte score of the kmeans solution: 0.09081949446607475


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
dist_mat = squareform(pdist(scaled_df, metric="euclidean"))
dist_mat = pd.DataFrame(dist_mat, columns=scaled_df.index, index=scaled_df.index)
dist_mat

clst = AgglomerativeClustering(n_clusters=2, affinity="cosine", linkage="average")
clst.fit(scaled_df)


scaled_df["Labels"] = clst.labels_
clst_avg = scaled_df.groupby("Labels").mean()
display(clst_avg.style.background_gradient())
# print(f"Heirarchical scaled centroid points with 2 clusters:\n {clst.labels_}")

# centroids = scaler.inverse_transform(clst.labels_)
# centroids_df = pd.DataFrame(centroids, columns=scaled_df.columns)
# print("\n Gauss unscaled centroid points with 2 clusters")
# display(centroids_df.style.background_gradient())

clusters = clst.fit_predict(scaled_df)


print(
    "Adjusted Rand Index of the Heirarchical solution: {}".format(
        metrics.adjusted_rand_score(y, clusters)
    )
)
print(
    "The silhoutte score of the Heirarchical solution: {}".format(
        metrics.silhouette_score(X, clusters, metric="euclidean")
    )
)

,0,1,2,3,4,5,6,7,8,9,10,11,12
Labels,,,,,,,,,,,,,
0,-0.255776,-0.043260,-0.345516,-0.057135,-0.076104,0.053498,-0.100719,0.465395,-0.462210,-0.382406,-0.363611,-0.284575,-0.277420
1,0.436189,0.073774,0.589228,0.097436,0.129784,-0.091233,0.171761,-0.793666,0.788233,0.652139,0.620087,0.485302,0.473100


Adjusted Rand Index of the Heirarchical solution: 0.22645337051449754
The silhoutte score of the Heirarchical solution: 0.08005124221301219


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Kmeans has the best scores.

### 2. GMM implementation of scikit-learn has a parameter called covariance_type. 
This parameter determines the type of covariance parameters to use. 
There are four types that you can specify:
1. full: This is the default. Each component has its own general covariance matrix.
2. tied: All components share the same general covariance matrix.
3. diag: Each component has its own diagonal covariance matrix.
4. spherical: Each component has its own single variance.

Try all of these. Which one performs best in terms of ARI and silhouette scores?

In [32]:
X = df.iloc[:, :13]
y = df.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace="?", value=0)

# Binarize `y` so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 0, 1)

scaler = StandardScaler()
scaled = scaler.fit_transform(X)
scaled_df = pd.DataFrame(scaled, columns=X.columns)
scaled_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.948726,0.686202,-2.251775,0.757525,-0.264900,2.394438,1.016684,0.017197,-0.696631,1.087338,2.274579,-0.711131,0.659133
1,1.392002,0.686202,0.877985,1.611220,0.760415,-0.417635,1.016684,-1.821905,1.435481,0.397182,0.649113,2.504881,-0.865426
2,1.392002,0.686202,0.877985,-0.665300,-0.342283,-0.417635,1.016684,-0.902354,1.435481,1.346147,0.649113,1.432877,1.167319
3,-1.932564,0.686202,-0.165268,-0.096170,0.063974,-0.417635,-0.996749,1.637359,-0.696631,2.122573,2.274579,-0.711131,-0.865426
4,-1.489288,-1.457296,-1.208521,-0.096170,-0.825922,-0.417635,1.016684,0.980537,-0.696631,0.310912,-0.976352,-0.711131,-0.865426


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
ctypes = ["full", "tied", "diag", "spherical"]
for i in ctypes:
    gauss = GaussianMixture(n_components=2, covariance_type=i)
    gauss.fit(scaled_df)

    # print(f"GMM with covariance type {i} means\n {gauss.means_}")

    centroids = scaler.inverse_transform(gauss.means_)
    centroids_df = pd.DataFrame(centroids, columns=scaled_df.columns)
    print(f"GMM with cov type {i}:")
    # display(centroids_df.style.background_gradient())

    clusters = gauss.fit_predict(scaled_df)

    print(
        "Adjusted Rand Index of the GMM solution: {}".format(
            metrics.adjusted_rand_score(y, clusters)
        )
    )
    print(
        "The silhoutte score of the GMM solution: {}".format(
            metrics.silhouette_score(X, clusters, metric="euclidean")
        )
    )

GMM with cov type full:
Adjusted Rand Index of the GMM solution: 0.18389186035089963
The silhoutte score of the GMM solution: 0.03924007666635825
GMM with cov type tied:
Adjusted Rand Index of the GMM solution: 0.18389186035089963
The silhoutte score of the GMM solution: 0.03924007666635825
GMM with cov type diag:
Adjusted Rand Index of the GMM solution: 0.3870024156200561
The silhoutte score of the GMM solution: 0.06867625547193433
GMM with cov type spherical:
Adjusted Rand Index of the GMM solution: 0.20765243525722465
The silhoutte score of the GMM solution: 0.047221690719502496


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Diag has the best scores.